In [1]:
import pandas as pd
import numpy as np

def process_observed_data(filename):
    data = pd.read_csv(filename, sep=',', header=None, names=['Timestamp', 'Y', 'X'])
    data['Timestamp'] = pd.to_datetime(data['Timestamp'], utc=True)
    
    observer_x, observer_y = 62, 62 - (18.8 / (80/62))  # Assume this is the observer's pixel location
    pixel_to_degrees = (80/62)  # Conversion factor from pixel to degrees
    
    positions = []
    for index, point in data.iterrows():
        dx, dy = point['X'] - observer_x, point['Y'] - observer_y
        radius = np.sqrt(dx**2 + dy**2) * pixel_to_degrees
        azimuth = np.degrees(np.arctan2(dx, dy))
        # Normalize the azimuth to ensure it's within 0 to 360 degrees
        azimuth = (azimuth + 360) % 360
        elevation = 90 - radius
        positions.append((point['Timestamp'], point['Y'], point['X'], elevation, azimuth))
    
    df_positions = pd.DataFrame(positions, columns=['Timestamp', 'Y', 'X', 'Elevation', 'Azimuth'])
    return df_positions

def main(filename):
    observed_positions = process_observed_data(filename)
    if not observed_positions.empty:
        print(observed_positions)
        observed_positions.to_csv('processed_observed_data.csv', index=False)
    else:
        print("No valid observed data found.")
    return observed_positions

if __name__ == "__main__":
    filename = 'white_pixel_coordinates_xor.csv'
    observed_positions = main(filename)
    observed_positions

                     Timestamp   Y   X  Elevation     Azimuth
0    2024-07-17 18:07:57+00:00  63  20  32.202407  290.340463
1    2024-07-17 18:07:58+00:00  63  20  32.202407  290.340463
2    2024-07-17 18:07:59+00:00  63  20  32.202407  290.340463
3    2024-07-17 18:08:00+00:00  62  21  33.855621  289.563476
4    2024-07-17 18:08:01+00:00  62  21  33.855621  289.563476
...                        ...  ..  ..        ...         ...
6995 2024-07-17 20:12:51+00:00  70  89  44.592287   50.106875
6996 2024-07-17 20:12:52+00:00  70  89  44.592287   50.106875
6997 2024-07-17 20:12:53+00:00  69  89  45.408854   51.379031
6998 2024-07-17 20:12:54+00:00  69  89  45.408854   51.379031
6999 2024-07-17 20:12:55+00:00  68  89  46.202625   52.697970

[7000 rows x 5 columns]


In [ ]:
from skyfield.api import load, wgs84, utc
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import math
import os

def load_data():
    stations_url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle'
    satellites = load.tle_file(stations_url)
    return satellites

def set_observation_time(year, month, day, hour, minute, second):
    ts = load.timescale()
    return ts.utc(year, month, day, hour, minute, second)

def process_observed_data(filename, start_time, merged_data_file):
    data = pd.read_csv(filename, sep=',', header=None, names=['Timestamp', 'Y', 'X'])
    data['Timestamp'] = pd.to_datetime(data['Timestamp'], utc=True)
    interval_start_time = pd.to_datetime(start_time, utc=True)
    interval_end_time = interval_start_time + pd.Timedelta(seconds=14)
    filtered_data = data[(data['Timestamp'] >= interval_start_time) & (data['Timestamp'] < interval_end_time)]
    if filtered_data.empty:
        print("No data found.")
        return None

    merged_data = pd.read_csv(merged_data_file, parse_dates=['Timestamp'])
    merged_data['Timestamp'] = pd.to_datetime(merged_data['Timestamp'], utc=True)
    merged_filtered_data = merged_data[(merged_data['Timestamp'] >= interval_start_time) & (merged_data['Timestamp'] < interval_end_time)]
    
    if merged_filtered_data.empty:
        print("No matching data found in merged_data_file.")
        return None

    if len(merged_filtered_data) < 3:
        print("Not enough data points in merged_filtered_data.")
        return None

    start_data = merged_filtered_data.iloc[0]
    middle_data = merged_filtered_data.iloc[len(merged_filtered_data)//2]
    end_data = merged_filtered_data.iloc[-2]
    rotation = -43
    positions = [
        (start_data['Timestamp'], (90 - start_data['Elevation'], (start_data['Azimuth'] + rotation) % 360)),
        (middle_data['Timestamp'], (90 - middle_data['Elevation'], (middle_data['Azimuth'] + rotation) % 360)),
        (end_data['Timestamp'], (90 - end_data['Elevation'], (end_data['Azimuth'] + rotation) % 360))
    ]
    
    return positions


def calculate_positions_for_satellite(satellite, observer_location, start_time, interval_seconds, step_seconds):
    ts = load.timescale()
    positions = []
    for second in range(0, interval_seconds + 1, step_seconds):
        current_time = start_time + timedelta(seconds=second)
        difference = satellite - observer_location
        topocentric = difference.at(current_time)
        alt, az, distance = topocentric.altaz()
        if alt.degrees > 20:
            positions.append((alt.degrees, az.degrees))
    return positions

def calculate_direction_vector(point1, point2):
    """Calculate the direction vector from point1 to point2."""
    alt_diff = point2[0] - point1[0]
    az_diff = azimuth_difference(point2[1], point1[1])
    magnitude = math.sqrt(alt_diff**2 + az_diff**2)
    return (alt_diff / magnitude, az_diff / magnitude) if magnitude != 0 else (0, 0)

def azimuth_difference(az1, az2):
    """Calculate the smallest difference between two azimuth angles."""
    diff = abs(az1 - az2) % 360
    if diff > 180:
        diff = 360 - diff
    return diff

def calculate_trajectory_distance(observed_positions, satellite_positions):
    """Calculate the distance measure between observed and satellite trajectories."""
    altitude_range = 90.0  # Maximum possible altitude difference
    azimuth_range = 180.0  # Maximum possible azimuth difference
    direction_range = 2.0  # Maximum possible direction difference (since vectors are normalized)
    
    distance = 0
    for i in range(len(observed_positions)):
        # Calculate distance between points
        alt_deviation = abs(observed_positions[i][0] - satellite_positions[i][0]) / altitude_range
        az_deviation = azimuth_difference(observed_positions[i][1], satellite_positions[i][1]) / azimuth_range
        distance += alt_deviation + az_deviation
    
    # Calculate the overall direction vectors
    obs_dir_vector = calculate_direction_vector(observed_positions[0], observed_positions[-1])
    sat_dir_vector = calculate_direction_vector(satellite_positions[0], satellite_positions[len(observed_positions) - 1])
    
    # Calculate direction difference
    direction_diff = math.sqrt((obs_dir_vector[0] - sat_dir_vector[0])**2 + (obs_dir_vector[1] - sat_dir_vector[1])**2) / direction_range
    
    # Add the direction difference to the distance measure
    total_distance = distance + direction_diff
    
    return total_distance

def find_matching_satellites(satellites, observer_location, observed_positions_with_timestamps):
    best_match = None
    closest_distance = float('inf')

    ts = load.timescale()
    
    for satellite in satellites:
        satellite_positions = []
        valid_positions = True
        
        for observed_time, observed_data in observed_positions_with_timestamps:
            # Calculate satellite position at the specific observed timestamp
            difference = satellite - observer_location
            topocentric = difference.at(ts.utc(observed_time.year, observed_time.month, observed_time.day, observed_time.hour, observed_time.minute, observed_time.second))
            alt, az, _ = topocentric.altaz()
            
            if alt.degrees <= 20:
                valid_positions = False
                break
            
            satellite_positions.append((alt.degrees, az.degrees))
        
        if valid_positions:
            total_distance = calculate_trajectory_distance(
                [(90 - data[0], data[1]) for _, data in observed_positions_with_timestamps], 
                satellite_positions
            )
            
            if total_distance < closest_distance:
                closest_distance = total_distance
                best_match = satellite.name
    
    return [best_match] if best_match else []

def calculate_distance_for_best_match(satellite, observer_location, start_time, interval_seconds):
    ts = load.timescale()
    distances = []
    for second in range(0, interval_seconds + 1):
        current_time = start_time + timedelta(seconds=second)
        difference = satellite - observer_location
        topocentric = difference.at(current_time)
        distance = topocentric.distance().km
        distances.append(distance)
    return distances

def main(filename, year, month, day, hour, minute, second, merged_data_file, satellites):
    initial_time = set_observation_time(year, month, day, hour, minute, second)
    observer_location = wgs84.latlon(latitude_degrees=51.053464, longitude_degrees=4.361142, elevation_m=60)
    observed_positions_with_timestamps = process_observed_data(filename, initial_time.utc_strftime('%Y-%m-%dT%H:%M:%SZ'), merged_data_file)
    if observed_positions_with_timestamps is None:
        return [], [], []

    matching_satellites = find_matching_satellites(satellites, observer_location, observed_positions_with_timestamps)
    if not matching_satellites:
        return observed_positions_with_timestamps, [], []

    best_match_satellite = next(sat for sat in satellites if sat.name == matching_satellites[0])
    distances = calculate_distance_for_best_match(best_match_satellite, observer_location, initial_time, 14)
    
    return observed_positions_with_timestamps, matching_satellites, distances

def process_intervals(filename, start_year, start_month, start_day, start_hour, start_minute, start_second, end_year, end_month, end_day, end_hour, end_minute, end_second, merged_data_file, satellites):
    results = []
    
    start_time = datetime(start_year, start_month, start_day, start_hour, start_minute, start_second, tzinfo=utc)
    end_time = datetime(end_year, end_month, end_day, end_hour, end_minute, end_second, tzinfo=utc)
    current_time = start_time
    
    while current_time <= end_time:
        print(f"Processing data for {current_time}")
        observed_positions_with_timestamps, matching_satellites, distances = main(filename, current_time.year, current_time.month, current_time.day, current_time.hour, current_time.minute, current_time.second, merged_data_file, satellites)
        if matching_satellites:
            for second in range(15):
                if second < len(distances):
                    results.append({
                        'Timestamp': current_time + timedelta(seconds=second),
                        'Connected_Satellite': matching_satellites[0],
                        'Distance': distances[second]
                    })
        current_time += timedelta(seconds=15)
    
    result_df = pd.DataFrame(results)
    return result_df

if __name__ == "__main__":
    filename = 'white_pixel_coordinates_xor.csv'
    merged_data_file = 'processed_observed_data.csv'
    
    # Load satellite data once
    satellites = load_data()
    
    # Process intervals for the specified range of timestamps
    result_df = process_intervals(filename, 2024, 7, 17,  18, 7,57, 2024, 7, 17,  20, 12,42, merged_data_file, satellites)

    # Load the data from both CSV files
    merged_data_df = pd.read_csv(merged_data_file, parse_dates=['Timestamp'])

    # Load the existing matched_satellite_data.csv if it exists
    if os.path.exists('matched_satellite_data.csv'):
        existing_df = pd.read_csv('matched_satellite_data.csv', parse_dates=['Timestamp'])
    else:
        existing_df = pd.DataFrame()

    # Merge the dataframes on Timestamp column
    merged_df = pd.merge(merged_data_df, result_df, on='Timestamp', how='inner')

    # Append the new data to the existing data
    updated_df = pd.concat([existing_df, merged_df]).drop_duplicates(subset=['Timestamp'], keep='last')

    # Save the updated dataframe to the CSV file
    updated_df.to_csv('matched_satellite_data.csv', index=False)

    print("Updated data saved to 'matched_satellite_data.csv'")


Processing data for 2024-07-17 18:07:57+00:00
Processing data for 2024-07-17 18:08:12+00:00
Processing data for 2024-07-17 18:08:27+00:00
Processing data for 2024-07-17 18:08:42+00:00
Processing data for 2024-07-17 18:08:57+00:00
Processing data for 2024-07-17 18:09:12+00:00
Processing data for 2024-07-17 18:09:27+00:00
Processing data for 2024-07-17 18:09:42+00:00
Processing data for 2024-07-17 18:09:57+00:00
Processing data for 2024-07-17 18:10:12+00:00
Processing data for 2024-07-17 18:10:27+00:00
Processing data for 2024-07-17 18:10:42+00:00
Processing data for 2024-07-17 18:10:57+00:00
Processing data for 2024-07-17 18:11:12+00:00
Processing data for 2024-07-17 18:11:27+00:00
Processing data for 2024-07-17 18:11:42+00:00
Processing data for 2024-07-17 18:11:57+00:00
Processing data for 2024-07-17 18:12:12+00:00
Processing data for 2024-07-17 18:12:27+00:00
Processing data for 2024-07-17 18:12:42+00:00
Processing data for 2024-07-17 18:12:57+00:00
Processing data for 2024-07-17 18: